In [9]:
#url_list 만들기

from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)
start_dt = date(2017, 1, 1)
end_dt = date(2017, 1, 1)

url_list = []
for dt in daterange(start_dt, end_dt):
    dtdt = dt.strftime("%Y%m%d")
    url_list.append("https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date={}".format(dtdt))    

In [10]:
#크롤링해서 JSON에 저장(문제는 중복된 프로그램명은 낮은 시청률을 반영한다는 것, 보수적 접근)

import urllib.request
from bs4 import BeautifulSoup
from collections import OrderedDict
import json

pn_list = []
pnss = OrderedDict()
for base_url in url_list:
    
    with urllib.request.urlopen(base_url) as url:
    
        doc = url.read()
        soup = BeautifulSoup(doc, "html.parser")
        names = soup.find_all("td", class_="tb_txt")
        percents10 = soup.find_all("td", class_="percent")
        percents20 = soup.find_all("td", class_="percent_g")
    
        name_list = []
        percent_list = []
        pnss["날짜"] = base_url[-8:]
        for name in names[0:20]:
            name_list.append(name.text.strip())
        for percent10 in percents10[0:10]:
            percent_list.append(percent10.text.strip())
        for percent20 in percents20[0:10]:
            percent_list.append(percent20.text.strip())
        for i in range(0,20):
            pnss[name_list[i]] = percent_list[i]
        

    pn_list.append(pnss)
    
with open('rating2017.json', 'w', encoding="utf-8") as make_file:
    json.dump(pn_list, make_file, ensure_ascii=False, indent="\t")

In [5]:
url_list

['https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date=20170101']